In [ ]:
import json
import pandas as pd
import numpy as np
import os

In [ ]:
import time
import requests

def pause_report(length, random_delay, file_count, disp = True):
    
    pause_time = length*np.random.normal(1,random_delay)
    if disp: print(f"Downloaded {file_count} images. Pausing scraper for {round(pause_time,2)} seconds.")
    time.sleep(pause_time)

    
def download_files(query, df, file_start, file_end, short_pause = 2, long_pause = 60, random_delay = 0.25):

    file_count = 0 #number of files downloaded
    alrdy_exists = 0 #number of files that already exist and are skipped over
    skipped_urls = {} #urls that can't be connected to - need to retry at a later time
    
    if not os.path.exists(f'full_data/images/query_{query}'):
        print(f'Creating directory: full_data/images/query_{query}')
        os.makedirs(f'full_data/images/query_{query}')
    
    for i in range(file_start, file_end):

        if i >= len(df):
            print(f"Index {i} does not exist in dataframe.")
            print("End of dataframe has probably been reached. Process is terminating.")
            return 0
        
        filename = f"full_data/images/query_{query}/{df['hash_id'][i]}.jpg" # name the image using its hash_id

        if not os.path.exists(filename): # skip over files that have been downloaded
            
            try:
                img_url = df['smaller_square_cover_url'][i]
                print(f'Downloading {img_url} as {filename}')
                
                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)
                
            except (requests.exceptions.RequestException or requests.exceptions.ConnectionError or AttributeError or TypeError) as e:
                print("Cannot establish connection with the following row in df:")
                print(f"{i}:{img_url}")
                print("Skipping to next row")
                skipped_urls[i] = img_url #add key value pair to skipped_urls dict
                
                if len(skipped_urls) > 10:
                    print("More than 10 urls have had connection error.")
                    print(skipped_urls)
                    pause_report(300, random_delay, file_count) # pause for 5 minutes if more than 10 urls with connection error
                       
        else: 
            print(f"{filename} already exists. Skipping to next image url in df.")
            alrdy_exists += 1
            
        # take a long pause if 100 images have been downloaded
        if file_count > 0 and file_count%100 == 0:
            pause_report(long_pause, random_delay, file_count)
            
        # take a long pause x 2 if 500 images have been downloaded
        if file_count > 0 and file_count%500 == 0:
            pause_report(long_pause*2, random_delay, file_count)
      
    # Retry skipped urls
    print(f"Skipped the following urls because failed to make a connection:")
    print(skipped_urls)
    
    if len(skipped_urls) > 0:
        print("Retrying skipped urls.")
        
        for index, url in skipped_urls.items():
            try:
                filename = f"full_data/images/query_{query}/{df['hash_id'][index]}.jpg"
                print(f'Downloading {url} as {filename}.')

                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)

            except Exception as e:
                print("Still unable to connect to skipped urls. Please manually check for error.")
                print(e.message, e.args)
                return 0
    
    print(f"{file_count} images downloaded.")
    print(f"Skipped {alrdy_exists} images because already exists in database.")
    print("Process finished.")
    return 1
    

In [ ]:
# load query
with open('query.txt', 'r') as f:
    query = f.read().strip()

In [ ]:
# convert the raw json files of a specific keyword to a dataframe
df = pd.read_json(f"full_data/query_{query}.json")

In [ ]:
# start the image download process
download_files(query, df, 0, 10_000)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(20,6))
for i in range(5):
    plt.subplot(1,5,i+1)
    img = mpimg.imread(f"images/query_{query}/{df['hash_id'][i]}.jpg")
    plt.title(f"{df['title'][i]}")
    plt.imshow(img)